In [3]:
import math
class ConvexHull:
    def __init__(self, points):
        self.hull = self.convex_hull_2d(points)
    
    def convex_hull_2d(self, points):
        if len(points) < 5:
            return self.find_convex_hull(points)
        else:
            midpoint = int(len(points)/2)
            left_hull = self.convex_hull_2d(points[:midpoint])
            right_hull = self.convex_hull_2d(points[midpoint:])

        return self.merge_hull(left_hull, right_hull)
        
    # Finds upper tangent of two polygons 'lower_hull' and 'upper_hull' represented as two vectors.
    def merge_hull(self, lower_hull, upper_hull):
        p = len(lower_hull)
        q = len(upper_hull)

        leftmost1 = lower_hull.index(max(lower_hull, key=lambda x: x[0]))
        leftmost2 = upper_hull.index(min(upper_hull, key=lambda x: x[0]))

        check_tangent = lambda p1, upper_hull, c: (upper_hull[1]-p1[1]) * (c[0]-upper_hull[0]) - (c[1]-upper_hull[1]) * (upper_hull[0]-p1[0])

        # find upper tangent
        i = next((i for i in range(p) if check_tangent(upper_hull[leftmost2], lower_hull[leftmost1], lower_hull[(leftmost1+1) % p]) >= 0), leftmost1)
        j = next((j for j in range(q) if check_tangent(lower_hull[i], upper_hull[leftmost2], upper_hull[(q+leftmost2-1) % q]) <= 0), leftmost2)

        uppera, upperb = i, j

        # find lower tangent
        j = next((j for j in range(p) if check_tangent(lower_hull[leftmost1], upper_hull[leftmost2], upper_hull[(leftmost2+1) % q]) >= 0), leftmost2)
        i = next((i for i in range(q) if check_tangent(upper_hull[j], lower_hull[leftmost1], lower_hull[(p+leftmost1-1) % p]) <= 0), leftmost1)

        lowera, lowerb = i, j

        final = []
        # if lowera < uppera:
        #     final = lower_hull[uppera:] + lower_hull[:lowera+1]
        # else:
        #     final = lower_hull[uppera:lowera+1]

        # ind = lowerb
        # final.append(upper_hull[lowerb])
        # while ind != upperb:
        #     ind = (ind+1) % q
        #     final.append(upper_hull[ind])

        # merge the two hulls *CHECK*
        if lowera < uppera:
            final = lower_hull[uppera:lowera+1] + upper_hull[lowerb:upperb+1]
        else:
            final = upper_hull[lowerb:upperb+1] + lower_hull[uppera:lowera+1]
        return final
        
    def find_convex_hull(self, hull):
        def find_corner(p1, q1):
            p = [p1[0] - reference_point[0], p1[1] - reference_point[1]]
            q = [q1[0] - reference_point[0], q1[1] - reference_point[1]]
            cross_product = p[0] * q[1] - p[1] * q[0]
            if cross_product > 0:
                return -1
            elif cross_product < 0:
                return 1
            else:
                return 0
    
        # Find the lower hull of the points

        s = set()
        n = len(hull)
        for i in range(n):
            for j in range(i+1, n):
                points = [hull[k] for k in range(n) if k != i and k != j]
                x1, y1 = hull[i]
                x2, y2 = hull[j]
                a, b, c = y1-y2, x2-x1, x1*y2-y1*x2
                pos = sum(1 for (x, y) in points if a*x+b*y+c >= 0)
                neg = sum(1 for (x, y) in points if a*x+b*y+c <= 0)
                if pos == n-2 or neg == n-2:
                    s.update({hull[i], hull[j]})

        final = [list(x) for x in s]

        # counterclockwise sort
        reference_point = [0, 0]
        n = len(final)
        reference_point = [sum(p[0] for p in final), sum(p[1] for p in final)]  # calculate reference point
        final = [[n*final[i][0], n*final[i][1]] for i in range(n)]

        from functools import cmp_to_key
        final = sorted(final, key=cmp_to_key(find_corner))
        for i in range(n):
            final[i] = [final[i][0]/n, final[i][1]/n]
        return final


In [4]:
if __name__ == '__main__':
    hull = ConvexHull(sorted([(1,2),(9,10),(19,10),(8,19),(4,6),(7,5)]))

    print('Convex Hull:')
    for x in hull.hull:
        print(int(x[0]), int(x[1]))

Convex Hull:
19 10
8 19
1 2
